In [1]:
from pexels_apis import PexelsAPI
import os
from dotenv import load_dotenv
import json
from react_agent.utils import extract_video_data
load_dotenv()

True

In [2]:
pexels = PexelsAPI(os.environ.get('PEXELS_API_KEY'))

In [3]:
search_params = {
    "query": "a person surrounded with water",
     
    "orientation": "portrait",
    "size": "medium",           
    "page": 1,                 
    "per_page": 5
}

videos = pexels.search_videos(search_params)
# json_output = json.dumps(videos, indent=4)
# print(json_output)
clean_data = extract_video_data(videos)
from pprint import pprint
pprint(clean_data)


[{'author': 'ROMAN ODINTSOV',
  'author_url': 'https://www.pexels.com/@roman-odintsov',
  'duration': 11,
  'height': 4096,
  'id': 8195680,
  'preview_image': 'https://images.pexels.com/videos/8195680/blooming-bright-color-dawn-8195680.jpeg?auto=compress&cs=tinysrgb&fit=crop&h=1200&w=630',
  'video_files': [{'file_type': 'video/mp4',
                   'fps': 29.969999313354492,
                   'height': 4096,
                   'link': 'https://videos.pexels.com/video-files/8195680/8195680-uhd_2160_4096_30fps.mp4',
                   'quality': 'uhd',
                   'size': 31569170,
                   'width': 2160},
                  {'file_type': 'video/mp4',
                   'fps': 29.969999313354492,
                   'height': 2048,
                   'link': 'https://videos.pexels.com/video-files/8195680/8195680-hd_1080_2048_30fps.mp4',
                   'quality': 'hd',
                   'size': 7655049,
                   'width': 1080},
                  {'file_

In [31]:
import re

def extract_video_name(url: str) -> str:
    """
    Extracts the descriptive name from a Pexels video URL.
    Example: "https://www.pexels.com/video/a-man-made-pond-surrounded-by-rocks-and-plants-8195680/"
             → "A Man Made Pond Surrounded By Rocks And Plants"
    """
    match = re.search(r'/video/([^/]+)-\d+/?$', url)
    if match:
        return match.group(1).replace('-', ' ').title()
    return "Untitled"

# Assume `clean_data` is your list of video dictionaries
video_dict = {
    str(video['id']): extract_video_name(video['video_url'])
    for video in clean_data
}

from pprint import pprint
pprint(video_dict)


{'17076996': 'The Sun Setting Over The Water On A Boat',
 '17949249': 'Sao Conrado Rj',
 '17982228': 'Woman Walking On The Sand Beach',
 '20257643': 'Two People Walking On The Beach At Sunset',
 '27989388': 'A Woman In A Gold Dress Posing On The Rocks',
 '28241358': 'A Woman Standing By The Water In A Brown Cardigan',
 '28557546': 'A Man Standing On The Beach Watching The Sunset',
 '30321490': 'Serene Beach Sunset With Calm Waves',
 '31879279': 'Scuba Divers Exploring Underwater Canyon',
 '31884015': 'Spring Walk Over Bridge In Urban Park Setting',
 '31892609': 'Scenic Lighthouse At Dusk With Sky Trail',
 '31930531': 'Serene Evening Swim In Coastal Waters',
 '4434370': 'A Person Walking On The Seashore',
 '4759582': 'Woman Walking On White Sand Of Sea Water',
 '4762777': 'Woman Walking Under The Water Falls',
 '4936970': 'Person Pose And Laying On A Tree Trunk In The Forest',
 '5010624': 'People Having A Vacation',
 '5057325': 'Motor Boats With Tourists At Venice City',
 '5201403': 'Wo

In [5]:
# pexels.get_video(5520073)

In [10]:
# The JSON object representing the script
script_data = {
    "title": "Why You Feel Like a Fraud—Even When You're Winning",
    "length": "90-120 seconds",
    "background_music": {
        "music": "subtle motivational"
    },
    "sections": [
        {
            "section": "HOOK",
            "text": "You crushed the presentation. Got the promotion. Nailed the project. But deep down, you're waiting for someone to tap you on the shoulder and say, 'We made a mistake.'",
            "visual": {
                "scene": "woman sitting at desk staring at a 'Employee of the Month' certificate",
                "camera_angle": "close-up of conflicted facial expression",
                "transition": "cut to",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": "1 second",
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "CONCEPT",
            "text": "That voice whispering, 'You don’t belong here'? It’s not imposter syndrome—it’s your brain replaying old tapes. Tapes from a time when your worth was measured by what you did, not who you are.",
            "visual": {
                "scene": "man walking alone in a park, looking down at his feet",
                "camera_angle": "side profile in moody lighting",
                "transition": "fade to black",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "REAL-WORLD EXAMPLE",
            "text": "You’re at a team meeting. Your idea gets praised. But instead of feeling proud, you think, 'They’re just being nice.' Later, you overhear a colleague say, 'She’s so confident.' You laugh bitterly. If only they knew. You replay every mistake, every hesitation, every moment you didn’t feel 'enough.' You work late—again—to prove you deserve the seat you’re already sitting in. But the more you achieve, the louder the doubt gets. It’s a loop: perform, doubt, overcompensate, repeat.",
            "visual": {
                "scene": "woman in a boardroom, colleagues clapping, but she looks uneasy",
                "camera_angle": "over-the-shoulder shot focusing on her discomfort",
                "transition": "cut to",
                "sound": {
                    "sound_effects": "faint office chatter",
                    "silence_duration": None,
                    "sound_effect_timing": "as the clapping starts"
                }
            }
        },
        {
            "section": "PSYCHOLOGICAL INSIGHT",
            "text": "This isn’t about your skills—it’s about internalized shame. You were conditioned to believe your value is tied to your output. So now, even when you succeed, your brain screams, 'Fraud!' because it doesn’t recognize worth without struggle.",
            "visual": {
                "scene": "person staring at their reflection in a window, looking lost",
                "camera_angle": "close-up of reflection with blurred background",
                "transition": "dissolve",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        },
        {
            "section": "ACTIONABLE TIP",
            "text": "Start a 'proof of worth' journal. Every day, write one thing you did—not because you had to, but because you chose to. Not for praise, but for joy. Slowly, you’ll rewire your brain to see your value beyond productivity.",
            "visual": {
                "scene": "person writing in a notebook at a cozy café table",
                "camera_angle": "overhead shot of hands writing",
                "transition": "fade to black",
                "sound": {
                    "sound_effects": "pen scratching paper",
                    "silence_duration": None,
                    "sound_effect_timing": "as the writing begins"
                }
            }
        },
        {
            "section": "CTA",
            "text": "Follow for more psychology that helps you silence the fraud narrative—and start owning your worth.",
            "visual": {
                "scene": "person walking confidently down a city street, sunlight breaking through clouds",
                "camera_angle": "wide shot with uplifting lighting",
                "transition": "cut to black",
                "sound": {
                    "sound_effects": None,
                    "silence_duration": None,
                    "sound_effect_timing": None
                }
            }
        }
    ]
}

In [11]:
from pathlib import Path
import requests
import json
from typing import Dict
from react_agent.structures import VideoScript
from react_agent.utils import extract_video_data  # Adjust path as needed
import re

def sanitize_filename(name: str) -> str:
    """
    Convert a string to a safe filename:
    - Replace spaces with underscores
    - Remove non-alphanumeric characters (except underscores and dashes)
    - Collapse multiple underscores
    """
    name = name.replace(" ", "_")
    name = re.sub(r"[^\w\-]", "", name)  # Keep alphanumerics, underscores, dashes
    name = re.sub(r"_+", "_", name)  # Collapse multiple underscores
    return name.strip("_")


async def get_videos_from_script(script_data: Dict) -> Dict:
    """Search Pexels for videos matching each visual scene and download them."""

    try:
        script_obj = VideoScript(**script_data)
    except Exception as e:
        return {"status": "error", "message": f"Invalid script data: {e}"}

    # Create a sanitized directory for the video set
    safe_title = sanitize_filename(script_obj.title)
    video_dir = Path(f"my_test_files/videos/{safe_title}/")

    video_dir.mkdir(parents=True, exist_ok=True)

    results = []

    for section in script_obj.sections:
        search_query = section.visual.scene

        search_params = {
            "query": search_query,
            "orientation": "portrait",
            "size": "medium",
            "page": 1,
            "per_page": 5,
        }

        pexels_response = pexels.search_videos(search_params)  # Use actual object

        if pexels_response.get("status_code") != 200:
            print(f"Failed to find video for: {search_query}")
            continue

        videos_data = extract_video_data(pexels_response)
        if not videos_data:
            print(f"No videos returned for: {search_query}")
            continue

        # print(f'videos data: {videos_data}\n')
        best_video = videos_data[0]
        target_aspect = 9 / 16

        # Filter to SD quality portrait-oriented videos
        sd_videos = [
            v for v in best_video["video_files"]
            if v["quality"] == "sd" and v["width"] / v["height"] <= target_aspect
        ]
        if not sd_videos:
            sd_videos = [v for v in best_video["video_files"] if v["quality"] == "sd"]

        if not sd_videos:
            print(f"No suitable SD video found for: {search_query}")
            continue

        download_video = min(
            sd_videos, key=lambda v: abs((v["width"] / v["height"]) - target_aspect)
        )
        print(f'download video: {download_video}\n')
        # Generate sanitized filename
        filename = sanitize_filename(f"{safe_title}_{section.section}") + ".mp4"
        video_path = video_dir /  filename

        try:
            response = requests.get(download_video["link"], stream=True)
            response.raise_for_status()

            with open(video_path, "wb") as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

            video_metadata = {
                "script_section": section.section,
                "pexels_id": best_video["id"],
                "author": best_video["author"],
                "author_url": best_video["author_url"],
                "video_url": best_video["video_url"],
                "download_url": download_video["link"],
                "file_path": str(video_path),
                "dimensions": f"{download_video['width']}x{download_video['height']}",
                "duration": best_video["duration"],
                "search_query": search_query,
                "attribution": f"Video by {best_video['author']} from Pexels",
            }

            results.append(video_metadata)

        except Exception as e:
            print(f"Failed to download video for {search_query}: {str(e)}")
            continue

    metadata_path = video_dir / "video_metadata.json"
    with open(metadata_path, "w") as f:
        json.dump(results, f, indent=4)

    return {
        "status": "success",
        "downloaded_videos": len(results),
        "total_sections": len(script_obj.sections),
        "video_directory": str(video_dir),
        "metadata_file": str(metadata_path),
        "videos": results,
    }


In [12]:
k = await get_videos_from_script(script_data)

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/8547568/8547568-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 567407}

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/6132796/6132796-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 961748}

download video: {'file_type': 'video/mp4', 'width': 360, 'height': 640, 'link': 'https://videos.pexels.com/video-files/8135918/8135918-sd_360_640_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 759532}

download video: {'file_type': 'video/mp4', 'width': 226, 'height': 426, 'link': 'https://videos.pexels.com/video-files/7207576/7207576-sd_226_426_25fps.mp4', 'quality': 'sd', 'fps': 25.0, 'size': 899090}

download video: {'file_type': 'video/mp4', 'width': 240, 'height': 408, 'link': 'https://videos.pexels.com/video-files/4460555/4460555-sd_240_408_25fps.mp4', 'quality': 'sd', 'fps'

In [13]:
k

{'status': 'success',
 'downloaded_videos': 6,
 'total_sections': 6,
 'video_directory': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning',
 'metadata_file': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/video_metadata.json',
 'videos': [{'script_section': 'HOOK',
   'pexels_id': 8547568,
   'author': 'Photo By: Kaboompics.com',
   'author_url': 'https://www.pexels.com/@karolina-grabowska',
   'video_url': 'https://www.pexels.com/video/a-tired-woman-looking-at-her-laptop-8547568/',
   'download_url': 'https://videos.pexels.com/video-files/8547568/8547568-sd_226_426_25fps.mp4',
   'file_path': 'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning_HOOK.mp4',
   'dimensions': '226x426',
   'duration': 19,
   'search_query': "woman sitting at desk staring at a 'Employee of the Month' certificate",
   'attribution': 'Video by Photo By: Kaboompics.com from Pexels'},
  {'script_sec

In [26]:
k['metadata_file']

'my_test_files/videos/Why_You_Feel_Like_a_FraudEven_When_Youre_Winning/video_metadata.json'

In [1]:
# import torch
# from ctc_forced_aligner import (
#     load_audio,
#     load_alignment_model,
#     generate_emissions,
#     preprocess_text,
#     get_alignments,
#     get_spans,
#     postprocess_results,
# )

# audio_path = "my_test_files/audios/one.wav"
# text_path = "my_test_files/audios/one.txt"
# language = "iso" # ISO-639-3 Language code
# device = "cuda" if torch.cuda.is_available() else "cpu"
# batch_size = 16


# alignment_model, alignment_tokenizer = load_alignment_model(
#     device,
#     dtype=torch.float16 if device == "cuda" else torch.float32,
# )

# audio_waveform = load_audio(audio_path, alignment_model.dtype, alignment_model.device)


# with open(text_path, "r") as f:
#     lines = f.readlines()
# text = "".join(line for line in lines).replace("\n", " ").strip()

# emissions, stride = generate_emissions(
#     alignment_model, audio_waveform, batch_size=batch_size
# )

# tokens_starred, text_starred = preprocess_text(
#     text,
#     romanize=True,
#     language=language,
# )

# segments, scores, blank_token = get_alignments(
#     emissions,
#     tokens_starred,
#     alignment_tokenizer,
# )

# spans = get_spans(tokens_starred, segments, blank_token)

# word_timestamps = postprocess_results(text_starred, spans, stride, scores)

# kokoro-onnx

In [3]:

import soundfile as sf
from misaki import en, espeak

from kokoro_onnx import Kokoro

# Misaki G2P with espeak-ng fallback
fallback = espeak.EspeakFallback(british=False)
g2p = en.G2P(trf=False, british=False, fallback=fallback)

# Kokoro
kokoro = Kokoro("/home/aditya-ladawa/Aditya/z_projects/short_creation/kokoro_files/kokoro.onnx", "/home/aditya-ladawa/Aditya/z_projects/short_creation/kokoro_files/voices-v1.0.bin")

# Phonemize
text = "AT the shoreline...... I found a way to swim !! In sands. Wait wait, .... Am I .....smart or what?"
phonemes, _ = g2p(text)

# Create
samples, sample_rate = kokoro.create(phonemes, "af_heart", is_phonemes=True)

# Save
sf.write("audio.wav", samples, sample_rate)
print("Created audio.wav")

Created audio.wav


In [3]:
# ! wget https://github.com/thewh1teagle/kokoro-onnx/releases/download/model-files-v1.0/voices-v1.0.bin

## FFMPEG

In [4]:
os.listdir('/home/aditya-ladawa/Aditya/z_projects/short_creation/src/react_agent/output_reels_v3/The_Hidden_Bias_That_Shapes_Reality_The_Observer-Expectancy_Effect/individual_reels')

['reel_ACTIONABLE_TIP.mp4',
 'reel_REAL-WORLD_EXAMPLE.mp4',
 'reel_HOOK.mp4',
 'reel_CTA.mp4',
 'reel_PSYCHOLOGICAL_INSIGHT.mp4',
 'reel_CONCEPT.mp4']

# FFMPG my code

In [1]:
import ffmpeg
import random
import os

# --- Configuration ---
BASE_VIDEOS_PATH = "/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/videos/" # Adjust as needed
OUTPUT_DIR_BASE = "output_reels_fades_ordered_v3" # New output folder

MIN_SEGMENT_DURATION = 3.0
MAX_SEGMENT_DURATION = 8.0
TRANSITION_DURATION = 0.5 # Used for inter-segment fades and final CTA fade-out
REEL_WIDTH = 480            # Lower resolution
REEL_HEIGHT = 854
OUTPUT_FPS = 24  

VIDEO_CODEC = 'libx264'
PRESET = 'ultrafast'
CRF = 35
VIDEO_BITRATE = '800k'
AUDIO_CODEC = 'aac'
AUDIO_BITRATE = '96k'

CRF = 23
PRESET_X264 = 'ultrafast'
XFADE_TRANSITIONS = ['fade']

SECTION_SILENCE = {
    'HOOK': 1.0,
    'CONCEPT': 1.3,
    'REAL-WORLD_EXAMPLE': 1.5,
    'PSYCHOLOGICAL_INSIGHT': 2.0,
    'ACTIONABLE_TIP': 1.2,
    'CTA': 0.8
}

# Define the desired order for final concatenation
SECTION_ORDER = ['HOOK', 'CONCEPT', 'REAL-WORLD_EXAMPLE', 'PSYCHOLOGICAL_INSIGHT', 'ACTIONABLE_TIP', 'CTA']


def get_duration(filename):
    try:
        probe = ffmpeg.probe(filename)
        return float(probe['format']['duration'])
    except Exception as e:
        print(f"Error probing duration for {filename}: {e}")
        return None


def apply_segment_effects(video_node):
    node = video_node.filter(
        'scale', w=f'iw*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)',
        h=f'ih*max({REEL_WIDTH}/iw,{REEL_HEIGHT}/ih)', eval='frame'
    )
    node = node.filter('crop', w=REEL_WIDTH, h=REEL_HEIGHT,
                       x=f'(iw-{REEL_WIDTH})/2', y=f'(ih-{REEL_HEIGHT})/2')
    node = node.filter('fps', fps=OUTPUT_FPS, round='near')
    node = node.filter('setpts', 'PTS-STARTPTS')
    return node


def create_reel_for_audio(audio_file_path, associated_video_files, output_file_path):
    section_name = os.path.splitext(os.path.basename(audio_file_path))[0]
    section_key = section_name.upper() # Key for SECTION_SILENCE and CTA check
    silence_duration = SECTION_SILENCE.get(section_key, 1.0)

    # 1. Measure audio
    audio_duration = get_duration(audio_file_path)
    if audio_duration is None:
        print(f"Skipping {audio_file_path}, could not get audio duration.")
        return None
    if not associated_video_files:
        print(f"Skipping {section_name} ({audio_file_path}), no associated video files.")
        return None

    target_duration = audio_duration + silence_duration
    print(f"\nProcessing {section_name}: audio {audio_duration:.2f}s + silence {silence_duration:.2f}s = target {target_duration:.2f}s")

    # 2. Collect segments until filling target_duration
    segments = []
    effective_video_duration = 0.0
    files_to_use = associated_video_files.copy()
    random.shuffle(files_to_use)
    max_segment_picking_attempts = len(files_to_use) * 3 + 5
    current_attempt_idx = 0
    file_picker_idx = 0

    while effective_video_duration < target_duration and current_attempt_idx < max_segment_picking_attempts:
        current_attempt_idx += 1
        if not files_to_use: break

        video_path = files_to_use[file_picker_idx % len(files_to_use)]
        source_video_duration = get_duration(video_path) or 0.0
        needed_effective_contribution = target_duration - effective_video_duration
        if needed_effective_contribution <= 0.01: break

        segment_raw_duration = 0.0
        is_first_segment = not bool(segments)
        min_raw_len_this_segment = MIN_SEGMENT_DURATION
        max_raw_len_this_segment = MAX_SEGMENT_DURATION

        if is_first_segment:
            ideal_len = min(needed_effective_contribution, max_raw_len_this_segment)
            segment_raw_duration = max(ideal_len, min_raw_len_this_segment if ideal_len >= min_raw_len_this_segment else ideal_len)
            segment_raw_duration = min(segment_raw_duration, source_video_duration)
        else:
            min_raw_len_this_segment = max(MIN_SEGMENT_DURATION, TRANSITION_DURATION + 0.01)
            target_raw_len_for_segment = needed_effective_contribution + TRANSITION_DURATION
            ideal_len = min(target_raw_len_for_segment, max_raw_len_this_segment)
            segment_raw_duration = max(ideal_len, min_raw_len_this_segment if ideal_len >= min_raw_len_this_segment else ideal_len)
            segment_raw_duration = min(segment_raw_duration, source_video_duration)

        valid_segment = True
        if segment_raw_duration < 0.1: valid_segment = False
        if not is_first_segment and segment_raw_duration <= TRANSITION_DURATION: valid_segment = False
        
        if not valid_segment:
            print(f"  Skipping {os.path.basename(video_path)} for {section_name}: calculated raw_t ({segment_raw_duration:.2f}s) unsuitable. Source_t: {source_video_duration:.2f}s.")
            file_picker_idx += 1
            continue

        start_time = random.uniform(0, max(0, source_video_duration - segment_raw_duration))
        print(f"  Segment for {section_name} from {os.path.basename(video_path)}: ss={start_time:.2f}, raw_t={segment_raw_duration:.2f}s")
        inp = ffmpeg.input(video_path, ss=start_time, t=segment_raw_duration)
        vid_node = apply_segment_effects(inp.video)
        segments.append({'stream': vid_node, 'raw_duration': segment_raw_duration})

        if is_first_segment:
            effective_video_duration += segment_raw_duration
        else:
            effective_video_duration += segment_raw_duration - TRANSITION_DURATION
        file_picker_idx += 1

    if not segments:
        print(f"Error: No video segments collected for {section_name} ({audio_file_path}). Target: {target_duration:.2f}s, Effective Found: {effective_video_duration:.2f}s")
        return None
    print(f"  Collected {len(segments)} segments for {section_name}. Effective video length: {effective_video_duration:.2f}s (target {target_duration:.2f}s)")

    # 3. Concatenate video segments with crossfades
    merged_video_stream = segments[0]['stream']
    timeline = segments[0]['raw_duration'] 
    for i in range(1, len(segments)):
        next_segment_data = segments[i]
        xfade_offset = max(0, timeline - TRANSITION_DURATION) 
        print(f"    Applying xfade to {section_name}: transition={XFADE_TRANSITIONS[0]}, duration={TRANSITION_DURATION:.2f}s, offset={xfade_offset:.2f}s")
        merged_video_stream = ffmpeg.filter(
            [merged_video_stream, next_segment_data['stream']], 'xfade',
            transition=XFADE_TRANSITIONS[0], duration=TRANSITION_DURATION, offset=xfade_offset
        ).filter('setpts', 'PTS-STARTPTS')
        timeline += next_segment_data['raw_duration'] - TRANSITION_DURATION
    print(f"  Video duration for {section_name} after xfades (before CTA fade): {timeline:.2f}s")

    # Apply fade-to-black for CTA section video stream
    if section_key == 'CTA':
        print(f"  Applying fade-to-black at the end of CTA section video.")
        # timeline is the actual duration of the video content for the CTA section
        fade_start_time = max(0, timeline - TRANSITION_DURATION)
        actual_fade_duration = timeline - fade_start_time
        
        if actual_fade_duration > 0.01: # Only apply if meaningful duration
            merged_video_stream = merged_video_stream.filter(
                'fade', type='out',
                start_time=fade_start_time,
                duration=actual_fade_duration,
                color='black' 
            )
            print(f"    CTA fade-out applied: start_time={fade_start_time:.2f}s, duration={actual_fade_duration:.2f}s. Video will end black.")
        else:
            print(f"    CTA video content ({timeline:.2f}s) too short for effective fade-out. Skipping fade for {section_name}.")

    # 4. Build padded audio
    tts_input = ffmpeg.input(audio_file_path)
    silence_input = ffmpeg.input(
        'anullsrc=channel_layout=stereo:sample_rate=44100', format='lavfi', t=silence_duration
    )
    padded_audio = ffmpeg.filter([tts_input.audio, silence_input.audio], 'concat', n=2, v=0, a=1)

    # 5. Merge video & audio for the section
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
    try:
        out_node = ffmpeg.output(
            merged_video_stream, padded_audio, output_file_path,
            vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC, audio_bitrate=AUDIO_BITRATE,
            crf=CRF, preset=PRESET_X264, movflags='+faststart',
        )
        print(f"  Writing section reel: {output_file_path} (target section duration: {target_duration:.2f}s)")
        ffmpeg.run(out_node, overwrite_output=True, quiet=False)
        print(f"Completed section: {output_file_path}")
        return output_file_path
    except ffmpeg.Error as e:
        print(f"Error writing section {output_file_path}: {e}")
        if e.stderr: print(f"FFmpeg stderr: {e.stderr.decode('utf8')}")
        return None
    except Exception as e_gen:
        print(f"General error during output for section {output_file_path}: {e_gen}")
        return None


def concatenate_sections(section_files, final_output):
    list_path = os.path.join(os.path.dirname(final_output), 'sections_to_concat.txt')
    with open(list_path, 'w') as f:
        for filepath in section_files:
            abs_filepath = os.path.abspath(filepath).replace('\\', '/')
            f.write(f"file '{abs_filepath}'\n")

    print(f"Concatenating {len(section_files)} sections into {final_output}")
    try:
        (
            ffmpeg
            .input(list_path, format='concat', safe=0)
            .output(
                final_output, vcodec=VIDEO_CODEC, acodec=AUDIO_CODEC,
                audio_bitrate=AUDIO_BITRATE, crf=CRF, preset=PRESET_X264,
                movflags='+faststart'
            )
            .run(overwrite_output=True, quiet=False)
        )
        print(f"Final reel created: {final_output}")
    except ffmpeg.Error as e:
        print(f"Error concatenating final reel: {e}")
        if e.stderr: print(f"FFmpeg stderr: {e.stderr.decode('utf8')}")
    except Exception as e_gen:
         print(f"General error during final concatenation: {e_gen}")


def main():
    os.makedirs(OUTPUT_DIR_BASE, exist_ok=True)
    if not os.path.isdir(BASE_VIDEOS_PATH):
        print(f"Error: Base videos path not found: {BASE_VIDEOS_PATH}")
        return

    scripts = [d for d in os.listdir(BASE_VIDEOS_PATH)
               if os.path.isdir(os.path.join(BASE_VIDEOS_PATH, d))]

    for script in scripts:
        base_script_path = os.path.join(BASE_VIDEOS_PATH, script)
        audio_dir = os.path.join(base_script_path, 'audio')
        visuals_root = os.path.join(base_script_path, 'visuals')
        output_script_root = os.path.join(OUTPUT_DIR_BASE, script)
        os.makedirs(output_script_root, exist_ok=True)

        if not os.path.isdir(audio_dir):
            print(f"Audio directory not found for script {script}: {audio_dir}. Skipping script.")
            continue
        print(f"\nProcessing Script folder: {script}")

        # Store paths to generated section reels, mapped by section key
        section_output_map = {} 
        
        # Process audio files to generate sections
        available_audio_files = sorted([f for f in os.listdir(audio_dir) 
                                     if f.lower().endswith(('.wav', '.mp3', '.m4a', '.aac'))])
        if not available_audio_files:
            print(f"No audio files found in {audio_dir} for script {script}. Skipping script.")
            continue

        for audio_file in available_audio_files:
            section_name_from_audio = os.path.splitext(audio_file)[0]
            section_key_from_audio = section_name_from_audio.upper() # For matching SECTION_ORDER

            vids_for_section = []
            section_visuals_folder = os.path.join(visuals_root, f'section_{section_name_from_audio}')
            
            potential_visual_sources = []
            if os.path.isdir(section_visuals_folder):
                potential_visual_sources.append(section_visuals_folder)
            if os.path.isdir(visuals_root):
                potential_visual_sources.append(visuals_root)

            for source_dir in potential_visual_sources:
                vids_for_section.extend([
                    os.path.join(source_dir, f)
                    for f in os.listdir(source_dir)
                    if f.lower().endswith(('.mp4', '.mov', '.avi', '.mkv', '.webm'))
                ])
            vids_for_section = sorted(list(set(vids_for_section)))

            audio_path = os.path.join(audio_dir, audio_file)
            safe_section_name = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in section_name_from_audio)
            output_section_reel_path = os.path.join(output_script_root, f'reel_{safe_section_name}.mp4')
            
            # The create_reel_for_audio function internally derives section_key for CTA fade
            result_path = create_reel_for_audio(audio_path, vids_for_section, output_section_reel_path)
            if result_path:
                section_output_map[section_key_from_audio] = result_path
        
        # After processing all audio files for the current script, prepare ordered list for concatenation
        ordered_section_files_for_concat = []
        print(f"\nPreparing final reel for script '{script}' based on order: {SECTION_ORDER}")
        for section_key_in_order in SECTION_ORDER:
            if section_key_in_order in section_output_map:
                ordered_section_files_for_concat.append(section_output_map[section_key_in_order])
                print(f"  + Added section '{section_key_in_order}'")
            else:
                print(f"  - Warning: Reel for section '{section_key_in_order}' is missing for script '{script}'. It will be excluded.")

        if ordered_section_files_for_concat:
            if len(ordered_section_files_for_concat) < len(SECTION_ORDER):
                 print(f"Warning: Concatenating {len(ordered_section_files_for_concat)} available sections for '{script}' (some were missing).")
            
            final_out_safe_script_name = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in script)
            final_reel_output_path = os.path.join(output_script_root, f'{final_out_safe_script_name}_final_ordered_reel.mp4')
            concatenate_sections(ordered_section_files_for_concat, final_reel_output_path)
        elif not available_audio_files:
             # This case is handled earlier by skipping the script.
             pass
        else: # Some audio files were processed, but none matched the required sections or failed.
            print(f"No sections were successfully processed and matched the defined order for script '{script}'. Final ordered reel will not be generated.")


if __name__ == '__main__':
    main()


Processing Script folder: The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger

Processing ACTIONABLE_TIP: audio 15.04s + silence 1.20s = target 16.24s
  Segment for ACTIONABLE_TIP from ACTIONABLE_TIP_6521502.mp4: ss=0.02, raw_t=8.00s
  Segment for ACTIONABLE_TIP from ACTIONABLE_TIP_7967230.mp4: ss=1.89, raw_t=8.00s
  Segment for ACTIONABLE_TIP from ACTIONABLE_TIP_8136217.mp4: ss=3.50, raw_t=1.24s
  Collected 3 segments for ACTIONABLE_TIP. Effective video length: 16.24s (target 16.24s)
    Applying xfade to ACTIONABLE_TIP: transition=fade, duration=0.50s, offset=7.50s
    Applying xfade to ACTIONABLE_TIP: transition=fade, duration=0.50s, offset=15.00s
  Video duration for ACTIONABLE_TIP after xfades (before CTA fade): 16.24s
  Writing section reel: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/reel_ACTIONABLE_TIP.mp4 (target section duration: 16.24s)


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed section: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/reel_ACTIONABLE_TIP.mp4

Processing CONCEPT: audio 9.86s + silence 1.30s = target 11.16s
  Segment for CONCEPT from CONCEPT_8860238.mp4: ss=15.33, raw_t=8.00s
  Segment for CONCEPT from CONCEPT_6100413.mp4: ss=19.87, raw_t=3.66s
  Collected 2 segments for CONCEPT. Effective video length: 11.16s (target 11.16s)
    Applying xfade to CONCEPT: transition=fade, duration=0.50s, offset=7.50s
  Video duration for CONCEPT after xfades (before CTA fade): 11.16s
  Writing section reel: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/reel_CONCEPT.mp4 (target section duration: 11.16s)


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed section: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/reel_CONCEPT.mp4

Processing CTA: audio 5.65s + silence 0.80s = target 6.45s
  Segment for CTA from CTA_8243250.mp4: ss=11.21, raw_t=6.45s
  Collected 1 segments for CTA. Effective video length: 6.45s (target 6.45s)
  Video duration for CTA after xfades (before CTA fade): 6.45s
  Applying fade-to-black at the end of CTA section video.
    CTA fade-out applied: start_time=5.95s, duration=0.50s. Video will end black.
  Writing section reel: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/reel_CTA.mp4 (target section duration: 6.45s)


...

ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Completed section: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/reel_REAL-WORLD_EXAMPLE.mp4

Preparing final reel for script 'The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger' based on order: ['HOOK', 'CONCEPT', 'REAL-WORLD_EXAMPLE', 'PSYCHOLOGICAL_INSIGHT', 'ACTIONABLE_TIP', 'CTA']
  + Added section 'HOOK'
  + Added section 'CONCEPT'
  + Added section 'REAL-WORLD_EXAMPLE'
  + Added section 'PSYCHOLOGICAL_INSIGHT'
  + Added section 'ACTIONABLE_TIP'
  + Added section 'CTA'
Concatenating 6 sections into output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger_final_ordered_reel.mp4


ffmpeg version 6.1.1-3ubuntu5 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13 (Ubuntu 13.2.0-23ubuntu3)
  configuration: --prefix=/usr --extra-version=3ubuntu5 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --disable-omx --enable-gnutls --enable-libaom --enable-libass --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libglslang --enable-libgme --enable-libgsm --enable-libharfbuzz --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-libzimg --ena

Final reel created: output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger_final_ordered_reel.mp4


[mp4 @ 0x5cbf4999f400] Starting second pass: moving the moov atom to the beginning of the file speed=18.2x    
[out#0/mp4 @ 0x5cbf498cbf40] video:23541kB audio:970kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.137006%
frame= 2020 fps=440 q=-1.0 Lsize=   24544kB time=00:01:24.12 bitrate=2390.1kbits/s dup=2 drop=0 speed=18.3x    
[libx264 @ 0x5cbf498f20c0] frame I:9     Avg QP:18.11  size: 66949
[libx264 @ 0x5cbf498f20c0] frame P:2011  Avg QP:20.46  size: 11687
[libx264 @ 0x5cbf498f20c0] mb I  I16..4: 100.0%  0.0%  0.0%
[libx264 @ 0x5cbf498f20c0] mb P  I16..4:  4.5%  0.0%  0.0%  P16..4: 47.6%  0.0%  0.0%  0.0%  0.0%    skip:47.9%
[libx264 @ 0x5cbf498f20c0] coded y,uvDC,uvAC intra: 46.9% 38.4% 11.5% inter: 26.8% 11.7% 1.9%
[libx264 @ 0x5cbf498f20c0] i16 v,h,dc,p: 47% 21% 17% 15%
[libx264 @ 0x5cbf498f20c0] i8c dc,h,v,p: 41% 20% 32%  8%
[libx264 @ 0x5cbf498f20c0] kb/s:2291.17
[aac @ 0x5cbf4998a200] Qavg: 22780.172


# faster-whisper

In [1]:
from pathlib import Path
import ffmpeg

# Path to the input video
reel = Path('/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger_final_ordered_reel.mp4')

# Generate the output audio file path
audiofilename = reel.with_suffix('.mp3')

# Create the ffmpeg input stream
input_stream = ffmpeg.input(str(reel))

# Extract the audio stream and save it as an MP3 file
ffmpeg.output(input_stream.audio, str(audiofilename)).overwrite_output().run(quiet=True)

print(audiofilename)


/home/aditya-ladawa/Aditya/z_projects/short_creation/my_test_files/output_reels_fades_ordered_v3/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger/The_Power_of_Silence_Why_Saying_Less_Makes_You_Stronger_final_ordered_reel.mp3


In [6]:
# from IPython.display import Audio

# Audio(audiofilename)

In [7]:
from faster_whisper import WhisperModel

model = WhisperModel('base', device="cpu", compute_type='int8')

In [26]:
segments, info = model.transcribe(audiofilename, word_timestamps=True)
segments = list(segments)  # The transcription will actually run here.
# for segment in segments:
#     for word in segment.words:
#         print("[%.2fs -> %.2fs] %s" % (word.start, word.end, word.word))

In [27]:

wordlevel_info = []

for segment in segments:
    for word in segment.words:
      wordlevel_info.append({'word':word.word,'start':word.start,'end':word.end})


In [6]:

# wordlevel_info

In [28]:
import json

# Save to JSON
with open('data.json', 'w') as f:
    json.dump(wordlevel_info, f, indent=4)

def split_text_into_lines(data):
    MaxChars = 30
    MaxDuration = 2.5
    MaxGap = 1.5

    subtitles = []
    line = []
    line_duration = 0

    for idx, word_data in enumerate(data):
        word = word_data["word"]
        start = word_data["start"]
        end = word_data["end"]

        line.append(word_data)
        line_duration += end - start

        temp = " ".join(item["word"].strip() for item in line)
        new_line_chars = len(temp)

        duration_exceeded = line_duration > MaxDuration
        chars_exceeded = new_line_chars > MaxChars

        if idx > 0:
            gap = word_data['start'] - data[idx - 1]['end']
            maxgap_exceeded = gap > MaxGap
        else:
            maxgap_exceeded = False

        if duration_exceeded or chars_exceeded or maxgap_exceeded:
            if line:
                subtitle_line = {
                    "word": " ".join(item["word"].strip() for item in line),
                    "start": line[0]["start"],
                    "end": line[-1]["end"],
                    "textcontents": line
                }
                subtitles.append(subtitle_line)
                line = []
                line_duration = 0

    # Handle the last line
    if line:
        subtitle_line = {
            "word": " ".join(item["word"].strip() for item in line),
            "start": line[0]["start"],
            "end": line[-1]["end"],
            "textcontents": line
        }
        subtitles.append(subtitle_line)

    return subtitles

# Run and print
linelevel_subtitles = split_text_into_lines(wordlevel_info)

for line in linelevel_subtitles:
    json_str = json.dumps(line, indent=4)
    # print(json_str)


### workign code 1 (somewhat)

In [ ]:
from moviepy import TextClip, CompositeVideoClip, ColorClip
import numpy as np
import os

def create_caption(textJSON, framesize, font=os.path.join('/home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf'),
                   color='white', highlight_color='yellow', stroke_color='black', stroke_width=1):
    word_clips = []
    xy_textclips_positions = []

    x_pos = 0
    y_pos = 0
    line_width = 0  # Total width of words in the current line
    frame_width = framesize[0]
    frame_height = framesize[1]

    x_buffer = frame_width * 1 / 10
    max_line_width = frame_width - 2 * (x_buffer)

    fontsize = int(frame_height * 0.075)  # 7.5% of video height

    for index, wordJSON in enumerate(textJSON['textcontents']):
        duration = wordJSON['end'] - wordJSON['start']
        word_clip = TextClip(text=wordJSON['word'], font=font, font_size=fontsize, color=color, stroke_color=stroke_color,
                             stroke_width=stroke_width)
        word_clip = word_clip.with_duration(textJSON['end']).with_start(0)

        word_clip_space = TextClip(text=" ", font=font, font_size=fontsize, color=color)
        word_clip_space = word_clip_space.with_duration(duration).with_start(wordJSON['start'])

        word_width, word_height = word_clip.size
        space_width, space_height = word_clip_space.size

        if line_width + word_width + space_width <= max_line_width:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos": x_pos,
                "y_pos": y_pos,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.with_position((x_pos, y_pos))
            word_clip_space = word_clip_space.with_position((x_pos + word_width, y_pos))

            x_pos = x_pos + word_width + space_width
            line_width = line_width + word_width + space_width
        else:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos + word_height + 10
            line_width = word_width + space_width

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos": x_pos,
                "y_pos": y_pos,
                "width": word_width,
                "height": word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.with_position((x_pos, y_pos))
            word_clip_space = word_clip_space.with_position((x_pos + word_width, y_pos))
            x_pos = word_width + space_width

        word_clips.append(word_clip)
        word_clips.append(word_clip_space)

    # Highlighting the words
    for highlight_word in xy_textclips_positions:
        word_clip_highlight = TextClip(text=highlight_word['word'], font=font, font_size=fontsize, color=highlight_color,
                                      stroke_color=stroke_color, stroke_width=stroke_width)
        word_clip_highlight = word_clip_highlight.with_duration(highlight_word['duration']).with_start(highlight_word['start'])
        word_clip_highlight = word_clip_highlight.with_position((highlight_word['x_pos'], highlight_word['y_pos']))

        word_clips.append(word_clip_highlight)

    return word_clips, xy_textclips_positions

In [ ]:
from moviepy import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip
input_video = VideoFileClip(reel)
frame_size = input_video.size

all_linelevel_splits=[]

for line in linelevel_subtitles:
  out_clips,positions = create_caption(line,frame_size)

  max_width = 0
  max_height = 0

  for position in positions:
    # print (out_clip.pos)
    # break
    x_pos, y_pos = position['x_pos'],position['y_pos']
    width, height = position['width'],position['height']

    max_width = max(max_width, x_pos + width)
    max_height = max(max_height, y_pos + height)

  color_clip = ColorClip(size=(int(max_width*1.1), int(max_height*1.1)),
                       color=(64, 64, 64))
  color_clip = color_clip.with_opacity(.6)
  color_clip = color_clip.with_start(line['start']).with_duration(line['end']-line['start'])

  # centered_clips = [each.set_position('center') for each in out_clips]

  clip_to_overlay = CompositeVideoClip([color_clip]+ out_clips)
  clip_to_overlay = clip_to_overlay.with_position("bottom")


  all_linelevel_splits.append(clip_to_overlay)

input_video_duration = input_video.duration


final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

# Set the audio of the final video to be the same as the input video
final_video = final_video.with_audio(input_video.audio)

# Save the final clip as a video file with the audio included
final_video.write_videofile("output.mp4", fps=24, codec="libx264", audio_codec="aac")

### working code 2

In [34]:
from moviepy import TextClip, CompositeVideoClip, ColorClip
import numpy as np


def create_caption(textJSON, framesize,font = "/home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf",color='white', highlight_color='yellow',stroke_color='black',stroke_width=1):
    wordcount = len(textJSON['textcontents'])
    full_duration = textJSON['end']-textJSON['start']

    word_clips = []
    xy_textclips_positions =[]

    x_pos = 0
    y_pos = 0
    line_width = 0  # Total width of words in the current line
    frame_width = framesize[0]
    frame_height = framesize[1]

    x_buffer = frame_width*1/10

    max_line_width = frame_width - 2 * (x_buffer)

    fontsize = int(frame_height * 0.075) #7.5 percent of video height

    space_width = ""
    space_height = ""

    for index,wordJSON in enumerate(textJSON['textcontents']):
      duration = wordJSON['end']-wordJSON['start']
      word_clip = TextClip(text=wordJSON['word'], font = font,font_size=fontsize, color=color,stroke_color=stroke_color,stroke_width=stroke_width).with_start(textJSON['start']).with_duration(full_duration)
      word_clip_space = TextClip(text=" ", font = font,font_size=fontsize, color=color).with_start(textJSON['start']).with_duration(full_duration)
      word_width, word_height = word_clip.size
      space_width,space_height = word_clip_space.size
      if line_width + word_width+ space_width <= max_line_width:
            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos,
                "y_pos": y_pos,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.with_position((x_pos, y_pos))
            word_clip_space = word_clip_space.with_position((x_pos+ word_width, y_pos))

            x_pos = x_pos + word_width+ space_width
            line_width = line_width+ word_width + space_width
      else:
            # Move to the next line
            x_pos = 0
            y_pos = y_pos+ word_height+10
            line_width = word_width + space_width

            # Store info of each word_clip created
            xy_textclips_positions.append({
                "x_pos":x_pos,
                "y_pos": y_pos,
                "width" : word_width,
                "height" : word_height,
                "word": wordJSON['word'],
                "start": wordJSON['start'],
                "end": wordJSON['end'],
                "duration": duration
            })

            word_clip = word_clip.with_position((x_pos, y_pos))
            word_clip_space = word_clip_space.with_position((x_pos+ word_width , y_pos))
            x_pos = word_width + space_width


      word_clips.append(word_clip)
      word_clips.append(word_clip_space)


    for highlight_word in xy_textclips_positions:

      word_clip_highlight = TextClip(text=highlight_word['word'], font = font,font_size=fontsize, color=highlight_color,stroke_color=stroke_color,stroke_width=stroke_width).with_start(highlight_word['start']).with_duration(highlight_word['duration'])
      word_clip_highlight = word_clip_highlight.with_position((highlight_word['x_pos'], highlight_word['y_pos']))
      word_clips.append(word_clip_highlight)

    return word_clips,xy_textclips_positions

In [38]:
from moviepy import TextClip, CompositeVideoClip, concatenate_videoclips,VideoFileClip, ColorClip

input_video = VideoFileClip(reel)
frame_size = input_video.size

all_linelevel_splits=[]

for line in linelevel_subtitles:
  out_clips,positions = create_caption(line,frame_size)

  max_width = 0
  max_height = 0

  for position in positions:
    # print (out_clip.pos)
    # break
    x_pos, y_pos = position['x_pos'],position['y_pos']
    width, height = position['width'],position['height']

    max_width = max(max_width, x_pos + width)
    max_height = max(max_height, y_pos + height)

  color_clip = ColorClip(size=(int(max_width*1.1), int(max_height*1.1)),
                       color=(64, 64, 64))
  color_clip = color_clip.with_opacity(.6)
  color_clip = color_clip.with_start(line['start']).with_duration(line['end']-line['start'])

  # centered_clips = [each.set_position('center') for each in out_clips]

  clip_to_overlay = CompositeVideoClip([color_clip] + out_clips)
  clip_to_overlay = clip_to_overlay.with_position(("center", "bottom"))



  all_linelevel_splits.append(clip_to_overlay)

input_video_duration = input_video.duration


final_video = CompositeVideoClip([input_video] + all_linelevel_splits)

# Set the audio of the final video to be the same as the input video
final_video = final_video.with_audio(input_video.audio)

# Save the final clip as a video file with the audio included
final_video.write_videofile("output.mp4", fps=24, codec="libx264", audio_codec="aac")

f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
27.0
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
0.0
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
45.5
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
0.0
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
45.5
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
0.0
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
36.0
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
0.0
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
32.5
left
f /home/aditya-ladawa/Aditya/z_projects/short_creation/fonts/Cal_Sans/CalSans-Regular.ttf
0.0
left
f /ho

MoviePy - Done.
MoviePy - Writing video output.mp4



frame_index:  17%|█▋        | 322/1928 [00:45<03:49,  7.01it/s, now=None]

KeyboardInterrupt: 

In [ ]:
async def add_captions(state: State) -> CaptionOutput:
    """Add captions to video and return structured output"""

    captioner = state.captioner

    print(f"\nCAPTIONER FROM STATE: {captioner}\n")

    # Get latest script and create safe filename
    latest_script_obj = state.scripts[-1]
    safe_title = sanitize_filename(latest_script_obj.title)
    
    # Create output directory if it doesn't exist
    output_dir = Path(os.path.join(OUTPUT_DIR_BASE, safe_title))
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Define file paths
    original_video = output_dir / f"{safe_title}.mp4"
    reel_audio = output_dir / f"{safe_title}.mp3"
    captioned_video = output_dir / f"CAPTIONED_{safe_title}.mp4"
    subtitles_json = output_dir / f"{safe_title}.json"

    # Ensure audio file exists (extract if needed)
    if not reel_audio.exists():
        input_stream = ffmpeg.input(str(original_video))
        audio_stream = ffmpeg.output(
            input_stream.audio, 
            str(reel_audio),
            acodec='libmp3lame',
            audio_bitrate='192k'
        )
        ffmpeg.run(audio_stream, overwrite_output=True)
    
    # Generate and process subtitles
    word_segments = await captioner.generate_subtitles(str(reel_audio))
    line_subtitles = await captioner.create_line_level_subtitles(word_segments)
    await captioner.save_subtitles_to_json(line_subtitles, str(subtitles_json))

    # Add captions to video
    await captioner.add_captions_to_video(
        video_path=str(original_video),
        subtitles=line_subtitles,
        output_path=str(captioned_video)
    )
    
    caption_output = CaptionOutput(
        captioned_video_path=str(captioned_video),
        subtitles_json_path=str(subtitles_json),
        original_video_path=str(original_video),
        message="Captioning completed successfully"
    )

    return {'caption_output': caption_output}

